<a href="https://colab.research.google.com/github/Profe-Jose-Burgos/C-43/blob/main/lematizador_hackton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('universal_tagset')
nltk.download('spanish_grammars')
nltk.download('tagsets')
nltk.download('stopwords') # faltaaaaa
nltk.download('omw-1.4') #faltaaaa

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package spanish_grammars to /root/nltk_data...
[nltk_data]   Unzipping grammars/spanish_grammars.zip.
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [2]:
import sys
!{sys.executable} -m pip list #para saber que tengo instalado

import sys
!{sys.executable} -m pip install tensorflow

import sys
!{sys.executable} -m pip install keras

import sys
!{sys.executable} -m pip install nltk

import sys
!{sys.executable} -m pip install nltk

Package                       Version
----------------------------- ----------------------
absl-py                       1.3.0
aeppl                         0.0.33
aesara                        2.7.9
aiohttp                       3.8.3
aiosignal                     1.3.1
alabaster                     0.7.12
albumentations                1.2.1
altair                        4.2.0
appdirs                       1.4.4
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
atari-py                      0.2.9
atomicwrites                  1.4.1
attrs                         22.2.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.11.0
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        5.0.1
blis                          0.7.9
bokeh                         2.3.3
branca

In [ ]:
import nltk
nltk.download("stem")

[nltk_data] Error loading stem: Package 'stem' not found in index


False

In [6]:
import json
import pickle as pkl
import numpy as np
import nltk
import keras
from nltk.stem import WordNetLemmatizer
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense,Dropout
from tensorflow.keras.optimizers import SGD
# =
# ()
#""
ignore_words = ["?","¿","!","¡"]
data_file = open('intents.json').read() 
intents  = json.loads(data_file)
words  = []
classes  = []
documents  = []
for intent in intents["intents"]:
  for pattern in intent["patterns"]:
    w = nltk.word_tokenize(pattern) # tokeniza
    words.extend(w)
    documents.append((w,intent["tag"]))
    if intent["tag"] not in classes : 
      classes.append(intent["tag"])  


['hola', 'buenos', 'dias', 'buenas', 'tardes', 'buenas', 'noches', 'como', 'estas', 'hay', 'alguien', 'ahi', '?', 'hey', 'saludos', 'que', 'tal', 'consultar', 'precios', 'comprar', 'un', 'auto', 'averiguar', 'precio', 'busco', 'un', 'coche', 'informacion', 'necesito', 'ayuda', 'con', 'me', 'puede', 'colaborar', 'busco', 'un', 'auto', 'que', 'carros', 'tienen', 'busqueda', 'directa', 'directo', 'de', 'inmediato', 'busqueda', 'inmediata', 'sedan', 'pickup', 'camioneta', 'coupe', 'deportivos', 'nissan', 'honda', 'toyota', 'ford', 'isuzu', 'audi', 'bmw', 'subaru', 'mitsubihi', 'corvette', 'recomendación', 'recomiendes', 'algo', 'necesito', 'recomendación', 'que', 'me', 'recomiendas', 'necesito', 'algunas', 'específicaciones', 'año', 'año', ':', 'del', 'año', '120', 'caballos', 'de', 'fuerza', 'caballos', 'de', 'fuerza', ':', 'potencia', 'de', ':', '12', '000', 'dólares', 'precio', ':', '30000', 'costo', ':', '25000', 'cosumo', ':', 'que', 'sea', 'económico', '50', 'kilómetros', 'por', 'gal

In [7]:
from numpy.lib.function_base import append
from operator import index
from nltk.stem import SnowballStemmer
steemer = SnowballStemmer('spanish')
  
words = [steemer.stem(w.lower()) for w in words if w not in ignore_words]
pkl.dump(words,open('words.pkl',"wb"))
pkl.dump(classes,open('classes.pkl',"wb"))

traning = []
output_empty = [0]*len(classes)
for doc in documents:
  bag = []
  patterns_words = doc[0]
  patterns_words = [steemer.stem(w.lower()) for w in patterns_words if w not in ignore_words]
  for palabra in words:
    bag.append(1) if palabra in patterns_words else bag.append(0)
  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1
  traning.append([bag,output_empty])
traning = np.array(traning,dtype=object)
x_train = list(traning[:,0])
y_train = list(traning[:,1])



In [9]:
model = Sequential()
model.add(Dense(128, input_shape=(len(x_train[0]),), activation='relu')) 
model.add(Dropout(0.5))

model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(len(y_train[0]),activation='softmax'))

sgd = SGD(lr = 0.01,decay = 1e-6,momentum = 0.9,nesterov = True)
 
model.compile(loss="categorical_crossentropy",optimizer=sgd,metrics=["accuracy"])

hist = model.fit(np.array(x_train),np.array(y_train),epochs= 300,batch_size=5,verbose=1)
model.save("chatbot_model_hackton.h5",hist)
print("modelo creado")

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/300
18/18 [==============================] - 1s 3ms/step - loss: 0.0000e+00 - accuracy: 0.0116
Epoch 2/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0233
Epoch 3/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0349
Epoch 4/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0465
Epoch 5/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0814
Epoch 6/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0465
Epoch 7/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0465
Epoch 8/300
18/18 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.7791       
Epoch 9/300
18/18 [==============================] - 0s 2ms/step - loss: nan - accuracy: 1.0000
Epoch 10/300
18/18 [==============================] - 0s 2ms/step - loss: nan - 

In [15]:
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [stemer.stem(word.lower())for word in sentence_words]
  return setence_words

def bow(sentence,words,show_details = True):
  sentence_word =   clean_up_sentence(sentence)
  bag = [0]*len(words)

  for i in sentence_words:
    for j,w in enumerate(word):
      if w == i:
        bag[j] = 1
        if show_details:
          print("encontrado en la bolsa: ",w)
  return np.array(bag)

def predict_class(sentence,model):
  p = bow(sentence,words,show_details = False)
  model.predict(np.array([p]))[0]

  ERROR_THRESHOLD= 0.25
  result = [[i,r] for i,r in enumerate(res) if r> ERROR_THRESHOLD]
  result.sort(key = lambda x: x[1],coin_reverse = True )
  return_list = []
  for r in result:
    return_list.append({"intent": classes[r[0]], "probability": str(r[1])}) 
  return return_list

import random
def get_responces(ints,intents_json):
  tag= ints[0]
  list_of_itents =(intents_json["intent"])
  for i in list_of_itents:
    if list("tag") == tag:
       result= random.choice(i["responses"]) 
    break
  return result 

def chatbot_response(text):
  ints = predict_class(text,model)
  res = get_responces(ints,intents)
  return res